In [4]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import (
    StuffDocumentsChain,
    LLMChain,
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
)

In [13]:
# Load the PDF file
loader = PyPDFLoader("./crime-and-punishment.pdf")
data = loader.load()

In [69]:
# Assuming the book is divided into 7 parts
parts = [[] for _ in range(7)]
current_part = -1

# Example code to simulate splitting into parts and chapters
for i, doc in enumerate(data):
    if "Part" in doc.page_content:
        current_part += 1
    parts[current_part].append(doc)

In [98]:
import textwrap

In [126]:
import re

def extract_chapters(pages):
    # Combine all page content into a single string
    full_text = ''.join(page.page_content for page in pages)
    
    # Clean the combined text
    cleaned_text = re.sub(r'Free eBooks at Planet eBook.com', '', full_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    chapter_positions = {}
    
    chapter_titles = re.findall(r'Chapter [IVXLCDM]+', cleaned_text)
    for i, title in enumerate(chapter_titles):
        start = cleaned_text.find(title)
        end = cleaned_text.find(chapter_titles[i + 1], start) if i + 1 < len(chapter_titles) else len(cleaned_text)
        chapter_positions[title] = (start, end)
    
    # Extract and print the text of each chapter
    chapters = {}
    for chapter, (start, end) in chapter_positions.items():
        chapter_text = cleaned_text[start:end].strip()
        chapters[chapter] = chapter_text
        # print(f"{chapter_text}\n")
    
    return chapters

# Example usage
first_part = parts[3]
chapters = extract_chapters(first_part)

# print(chapters)
# Access specific chapter text
chapter_i_text = chapters['Chapter III']
wrapped_text = textwrap.fill(chapter_i_text, width=100)
print(wrapped_text)


Chapter III The fact was that up to the last moment he had never ex - pected such an ending; he had
been overbearing to the last degree, never dreaming that two destitute and defence - less women
could escape from his control. This conviction was strengthened by his vanity and conceit, a conceit
to the point of fatuity. Pyotr Petrovitch, who had made his way up from insignificance, was morbidly
given to self-admiration, had the highest opinion of his intelligence and capacities, and sometimes
even gloated in solitude over his image in the glass. But what he loved and valued above all was the
money he had amassed by his labour, and by all sorts of de - vices: that money made him the equal of
all who had been his superiors. When he had bitterly reminded Dounia that he had de - cided to take
her in spite of evil report, Pyotr Petrovitch had spoken with perfect sincerity and had, indeed,
felt genuine - ly indignant at such ‘black ingratitude.’ And yet, when he made Dounia his offer, he
was

In [127]:
import os

from groq import Groq

client = Groq(
    api_key='',
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "As a professional summarizer, create a concise and comprehensive summary of the provided text:\\n" + chapter_i_text,
        }
    ],
    temperature=0.5,
    # model="gemma-7b-it",
    model="llama3-8b-8192",
    max_tokens=4024,
)

print(chat_completion.choices[0].message.content)

Here is a concise and comprehensive summary of the provided text:

Pyotr Petrovitch Luzhin, who had been pursuing Dounia, is rejected by her after she discovers his true nature. Dounia's mother, Pulcheria Alexandrovna, is relieved and grateful that their financial troubles may be over, thanks to a surprise inheritance of 3,000 rubles. Meanwhile, Raskolnikov, who had been pushing Dounia to reject Luzhin, seems indifferent to the outcome. Razumihin, who has been watching over the family, is delighted by the turn of events and proposes a business venture to Raskolnikov and Dounia. However, Raskolnikov suddenly departs, leaving his family and friends behind, citing his own darkness and illness.

As Raskolnikov leaves, he has a brief and intense conversation with Razumihin, which hints at a deeper understanding between them. Razumihin is left feeling uneasy and concerned for Raskolnikov's well-being. The novel ends with Razumihin returning to the ladies, trying to reassure them that Raskoln

In [117]:
from openai import OpenAI

client = OpenAI(api_key="")

response = client.chat.completions.create(
    model="gpt-4-turbo",
    temperature=0.5,
    max_tokens=3500,
    messages=[{"role": "user", "content": "As a professional summarizer, create a concise and comprehensive summary of the provided text:\\n" + chapter_i_text}],
)

print(response.choices[0].message.content)



In Chapter I of "Crime and Punishment," Raskolnikov encounters Svidrigaïlov, a man he is initially suspicious of and bewildered by. Svidrigaïlov expresses a desire to acquaint himself with Raskolnikov, praising him and expressing a wish to assist Raskolnikov's sister, Avdotya Romanovna, whom he claims to care for despite her prejudices against him. Raskolnikov is dismissive and skeptical of Svidrigaïlov's intentions.

As their conversation unfolds, Svidrigaïlov discusses various personal matters, including his past relationships and actions which he justifies or questions in moral and philosophical terms. He reveals a complex character, oscillating between self-awareness and justification of his morally ambiguous actions.

Svidrigaïlov also hints at supernatural experiences, claiming to have been visited by the ghost of his deceased wife, Marfa Petrovna, which adds a layer of mystery and suggests his disturbed mental state.

The dialogue delves into themes of morality, the influence of

In [129]:
api_key=''

In [5]:
import os
import re
import textwrap
import time
from openai import OpenAI

# from groq import Groq

def summarize_text(text, client):
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            temperature=0.5,
            max_tokens=3500,
            messages=[{"role": "user", "content": "As a professional summarizer, create a concise and comprehensive summary of the provided text:\\n" + text}],
        )
        print('Chat GPT response sent')
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return ""

def extract_chapters(pages):
    full_text = ''.join(page.page_content for page in pages)
    cleaned_text = re.sub(r'Free eBooks at Planet eBook.com', '', full_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    chapter_positions = {}

    chapter_titles = re.findall(r'Chapter [IVXLCDM]+', cleaned_text)
    for i, title in enumerate(chapter_titles):
        start = cleaned_text.find(title)
        end = cleaned_text.find(chapter_titles[i + 1], start) if i + 1 < len(chapter_titles) else len(cleaned_text)
        chapter_positions[title] = (start, end)

    chapters = {}
    for chapter, (start, end) in chapter_positions.items():
        chapter_text = cleaned_text[start:end].strip()
        chapters[chapter] = chapter_text

    return chapters

def save_summary_to_file(summary, chapter_title, part_index):
    chapter_file_name = f"chapter_{part_index + 1}_{chapter_title.replace(' ', '_').lower()}.txt"
    try:
        with open(chapter_file_name, "w") as chapter_file:
            chapter_file.write(summary)
            # chapter_file.write("\n\nOriginal Text:\n\n")
            # chapter_file.write(textwrap.fill(chapter_text, width=100))
    except Exception as e:
        print(f"Error saving summary to file: {e}")

def save_master_summary_to_file(chapter_title, summary, master_summary_file):
    master_summary_file.write(f"Chapter: {chapter_title}\nSummary:\n{summary}\n\n")

def summarize_and_save_chapters_from_pdf(pdf_file_path):
    try:
        client = OpenAI(api_key="")
        master_summary_file = open("master_summary.txt", "a")

        loader = PyPDFLoader(pdf_file_path)
        data = loader.load()
        if not data:
            raise ValueError("PDF file could not be loaded or is empty")
        # print('PDF loaded')
        parts = [[] for _ in range(7)]
        current_part = -1

        for i, doc in enumerate(data):
            if "Part" in doc.page_content:
                current_part += 1
            parts[current_part].append(doc)
        print('Parts loaded')
        for i, part in enumerate(parts):
            chapters = extract_chapters(part)
            # print(f'Part {i + 1} chapters extracted Chapters: {chapters}')
            for chapter_title, chapter_text in chapters.items():
                print(f'Sleeping for 30 seconds')
                time.sleep(30)
                summary = summarize_text(chapter_text, client)
                print(f'Part {i + 1} Chapter {chapter_title}: {summary}')
                save_summary_to_file(summary, chapter_title, i)
                # print(f'Part {i + 1} Chapter {chapter_title} saved')
                save_master_summary_to_file(chapter_title, summary, master_summary_file)
        
        master_summary_file.close()
    except Exception as e:
        print(f"An error occurred: {e}")


pdf_path = "./crime-and-punishment.pdf"
summarize_and_save_chapters_from_pdf(pdf_path)


Parts loaded
Sleeping for 30 seconds
Chat GPT response sent
Part 1 Chapter Chapter I: On a sweltering early July evening, a young man named Raskolnikov emerges from his cramped, attic room in S. Place, cautiously avoiding his landlady to whom he owes rent. Plagued by debts and a growing sense of isolation and hypochondria, he dreads any human interaction, including trivial conversations that could force him to lie or make excuses. Despite his fears and the oppressive city environment of St. Petersburg, which intensifies his anxiety, Raskolnikov is introspective about his cowardice and lack of action. He contemplates a vague, sinister plan, reflecting on his own fears and the nature of human fear in general.

As he walks, Raskolnikov is disgusted by his surroundings and his own destitute appearance, which makes him feel particularly vulnerable when he encounters people from his past. His journey leads him to the apartment of an old pawnbroker, Alyona Ivanovna. He plans to pawn a watch b

In [6]:
# summarize the master summary.txt

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter


loader = TextLoader('master_summary.txt')
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)
print(f"Number of chunks: {len(docs)}")

Created a chunk of size 1050, which is longer than the specified 1000
Created a chunk of size 1097, which is longer than the specified 1000


Number of chunks: 83


In [ ]:
import os
import re
import textwrap
import time

from groq import Groq

def summarize_and_save_chapters(parts, api_key):
    client = Groq(api_key=api_key)
    time.sleep(30)  # Delay of 30 seconds between each request
    def summarize_text(text):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": "As a professional summarizer, create a concise and comprehensive summary of the provided text:\\n" + text,
                }
            ],
            temperature=0.2,
            model="llama3-8b-8192",
            max_tokens=4024,
        )
        return chat_completion.choices[0].message.content

    def extract_chapters(pages):
        full_text = ''.join(page.page_content for page in pages)
        cleaned_text = re.sub(r'Free eBooks at Planet eBook.com', '', full_text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

        chapter_positions = {}

        chapter_titles = re.findall(r'Chapter [IVXLCDM]+', cleaned_text)
        for i, title in enumerate(chapter_titles):
            start = cleaned_text.find(title)
            end = cleaned_text.find(chapter_titles[i + 1], start) if i + 1 < len(chapter_titles) else len(cleaned_text)
            chapter_positions[title] = (start, end)

        chapters = {}
        for chapter, (start, end) in chapter_positions.items():
            chapter_text = cleaned_text[start:end].strip()
            chapters[chapter] = chapter_text

        return chapters

    master_summary_file = open("master_summary.txt", "a")

    for i, part in enumerate(parts):
        chapters = extract_chapters(part)
        for chapter_title, chapter_text in chapters.items():
            summary = summarize_text(chapter_text)
            chapter_file_name = f"chapter_{i + 1}_{chapter_title.replace(' ', '_').lower()}.txt"
            with open(chapter_file_name, "w") as chapter_file:
                chapter_file.write(summary)
                chapter_file.write("\n\nOriginal Text:\n\n")
                chapter_file.write(textwrap.fill(chapter_text, width=100))
            master_summary_file.write(f"Chapter: {chapter_title}\nSummary:\n{summary}\n\n")
            
    
    master_summary_file.close()
